In [1]:
import pandas

## Load data

In [2]:
raw_path = '../../data/CSV/SolicitudesMadrid.csv'
solicitudes = pandas.read_csv(raw_path, sep=';')

## Score Interpretation

In [3]:
solicitudes['HerMat'] = (solicitudes['puntuacion_hermanos_matriculados'].map
                         ({0: 'Cero', 15: 'Uno', 30: 'Dos o más'}))
solicitudes['Dom'] = solicitudes['puntuacion_domicilio'].map({0: 'Extracomunitario', 8: 'Comunidad de Madrid', 12: 'Mismo municipio', 13: 'Mismo distrito'})
solicitudes['RenMin'] = solicitudes['puntuacion_renta_minima'].map({2: 'Sí', 0: 'No'})
solicitudes['TutTra'] = solicitudes['puntuacion_tutor_trabajando'].map({10: 'Sí', 0: 'No'})
solicitudes['FamNum'] = solicitudes['puntuacion_familia_numerosa'].map({11: 'Especial', 10: 'General', 0: 'No'})
solicitudes['ParMul'] = solicitudes['puntuacion_parto_multiple'].map({3: 'Sí', 0: 'No'})
solicitudes['FamMon'] = solicitudes['puntuacion_familia_monoparental'].map({3: 'Sí', 0: 'No'})
solicitudes['Acog'] = solicitudes['puntuacion_acogida'].map({3: 'Sí', 0: 'No'})
solicitudes['Dis'] = solicitudes['puntuacion_discapacidad'].map({7: 'Sí', 0: 'No'})
solicitudes['VioGen'] = solicitudes['puntuacion_violencia_genero'].map({2: 'Sí', 0: 'No'})
solicitudes['AntAlu'] = solicitudes['puntuacion_antiguo_alumno'].map({4: 'Sí', 0: 'No'})
solicitudes['Cen'] = solicitudes['puntuacion_criterio_centro'].map({3: 'Se cumple', 0: 'No se cumple'})

solicitudes_bachillerato = solicitudes[solicitudes['curso'].str.contains('Bachillerato')].copy()
solicitudes_no_artes = solicitudes_bachillerato[~solicitudes_bachillerato['curso'].str.contains('Artes', case=False, na=False)]
puntuacion_no_artes = solicitudes_no_artes.groupby('estudiante')['puntuacion_nota_media'].first()
solicitudes_bachillerato['puntuacion_nota_media_no_artes'] = solicitudes_bachillerato['estudiante'].map(puntuacion_no_artes)

def expediente_map(solicitud):
 curso = solicitud['curso']
 no_artes_map = {
  11: '>=9',
  9: '[8,9)',
  7: '[7,8)',
  6: '[6,7)',
  0: '<6',
 }
 if pandas.notna(solicitud['puntuacion_nota_media_no_artes']):
  nota = no_artes_map.get(solicitud['puntuacion_nota_media_no_artes'])
  if 'Artes' in curso and solicitud['puntuacion_nota_media'] > solicitud['puntuacion_nota_media_no_artes']:
   return f"{nota} + artes {solicitud['puntuacion_nota_media'] - solicitud['puntuacion_nota_media_no_artes']}"
  else:
   return nota
 else:
  artes_map = {
   0: '<6',
   0.5: '<6 + artes 0.5',
   1: '<6 + artes 1',
   1.5: '<6 + artes 1.5',
   6: '[6,7)',
   6.5: '[6,7) + artes 0.5',
   7: '[6,7) + artes 1 o [7,8)',
   7.5: '[6,7) + artes 1.5 o [7,8) + artes 0.5',
   8: '[7,8) + artes 1',
   8.5: '[7,8) + artes 1.5',
   9: '[8,9)',
   9.5: '[8,9) + artes 0.5',
   10: '[8,9) + artes 1',
   10.5: '[8,9) + artes 1.5',
   11: '>=9',
   11.5: '>=9 + artes 0.5',
   12: '>=9 + artes 1',
   12.5: '>=9 + artes 1.5'
  }
  return artes_map.get(solicitud['puntuacion_nota_media'])

solicitudes_bachillerato['NotMed'] = solicitudes_bachillerato.apply(expediente_map, axis=1)
solicitudes_bachillerato.drop(columns=['puntuacion_nota_media_no_artes'], inplace=True)

solicitudes_no_bachillerato = solicitudes[~solicitudes['curso'].str.contains('Bachillerato')].copy()
solicitudes_no_bachillerato['NotMed'] = 'Sin información'

solicitudes = pandas.concat([solicitudes_bachillerato, solicitudes_no_bachillerato], ignore_index=True)

interpreted_path = '../../data/Pandas/SolicitudesMadridInterpretadas.csv'
criterios = {
 'puntuacion_hermanos_matriculados': 'HerMat',
 'puntuacion_domicilio': 'Dom',
 'puntuacion_renta_minima': 'RenMin',
 'puntuacion_tutor_trabajando': 'TutTra',
 'puntuacion_familia_numerosa': 'FamNum',
 'puntuacion_parto_multiple': 'ParMul',
 'puntuacion_familia_monoparental': 'FamMon',
 'puntuacion_acogida': 'Acog',
 'puntuacion_discapacidad': 'Dis',
 'puntuacion_violencia_genero': 'VioGen',
 'puntuacion_antiguo_alumno': 'AntAlu',
 'puntuacion_criterio_centro': 'Cen',
 'puntuacion_nota_media': 'NotMed'
}

for puntuacion, circunstancia in criterios.items():
 idx = solicitudes.columns.get_loc(puntuacion)
 nueva = solicitudes.pop(circunstancia)
 solicitudes.insert(idx, circunstancia, nueva)

solicitudes.drop(columns=criterios.keys(), inplace=True)
solicitudes.drop(columns=['provincia_centro', 'municipio_centro', 'distrito_centro', 'nombre_distrito_centro', 'nombre_distrito_baremo', 'provincia_centro_adm', 'municipio_centro_adm', 'distrito_centro_adm', 'nombre_distrito_centro_adm'], inplace=True)
solicitudes.fillna(" ", inplace=True)
solicitudes.to_csv(interpreted_path, sep=';', index=False)